<a href="https://colab.research.google.com/github/VISHALGAURAV95/Info_extractor/blob/main/info_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [46]:
import os
import re
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader

# Specify the directory containing the PDF files
directory = 'content'  # Update this path as per your environment

all_text = []

# Process each PDF file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(directory, filename)
        print(f"Processing file: {pdf_path}")  # Debug: Check file being processed
        with open(pdf_path, 'rb') as src_file:
            reader = PdfReader(src_file)
            num_pages = len(reader.pages)
            print(f"Number of pages: {num_pages}")  # Debug: Check number of pages

            # Define the range of pages to process
            start_pno = 2
            end_pno = num_pages - 1

            # Extract text from the specified range of pages
            for pg in range(start_pno, end_pno):
                page = reader.pages[pg]
                text = page.extract_text()
                if text:
                    all_text.append(text.strip())  # Strip any leading/trailing whitespace

# Combine all extracted text
full_text = "\n".join(all_text)

# Split the text based on a pattern (assuming 'House No' starts a new record)
strn = re.split(r'\n(?=\s*House\s+No\s*:)', full_text)

# Debug: Check number of split sections
print(f"Number of sections after splitting: {len(strn)}")

# Extract details using regex and store in a list of dictionaries
rows = []
for eachline in strn:
    h_no = re.findall(r'House\s+No\s*:\s*([^\n]*)', eachline)
    gender_with_voterid = re.findall(r'(Female|Male)\s+Age\s*:\s*([^\n]+)', eachline)
    name = re.findall(r'Name\s*:\s*([^\n]*?)\s+(?:Father\'s\s+Name|Husband\'s\s+Name)\s*:', eachline)
    age = re.findall(r'Age\s*:\s*(\d+)', eachline)
    f_name = re.findall(r'(?:Father\'s\s+Name|Husband\'s\s+Name)\s*:\s*([^\n]*)', eachline)

    # Extract gender and voter ID
    gender = None
    voter_id = None
    if gender_with_voterid:
        gender = gender_with_voterid[0][0].strip()
        voter_id = gender_with_voterid[0][1].strip()

 # Extract age from the end of the line if not found in typical 'Age:' field
    if not age:
        age = re.findall(r'\b(\d{2})\b', eachline.strip())

    if h_no:
        row = {
            'House No': h_no[0].strip() if h_no else None,
            'Gender': gender,
            'Name': name[0].strip() if name else None,
            'Age': age[0].strip() if age else None,
            "Father's/Husband's Name": f_name[0].strip() if f_name else None,
            'Voter Id': voter_id
        }
        rows.append(row)

# Debug: Check the number of rows extracted
print(f"Number of rows extracted: {len(rows)}")

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(rows)

# Clean the DataFrame
df.replace('', np.nan, inplace=True)
df.dropna(how='all', inplace=True)

# Debug: Display the DataFrame
print("DataFrame preview:")
print(df.head())

# Save the DataFrame to an Excel file
output_path = 'output.xlsx'
df.to_excel(output_path, index=False)
print(f"Data saved to {output_path}")


Processing file: content/AC027PART003.pdf
Number of pages: 41
Processing file: content/AC027PART007.pdf
Number of pages: 47
Processing file: content/AC027PART006.pdf
Number of pages: 36
Processing file: content/AC027PART002.pdf
Number of pages: 28
Processing file: content/AC027PART004.pdf
Number of pages: 33
Processing file: content/AC027PART001.pdf
Number of pages: 24
Number of sections after splitting: 5599
Number of rows extracted: 5598
DataFrame preview:
                           House No  Gender            Name Age  \
0  Gender : Female Age : HGS0073544  Female         TIANARO  42   
1    Gender : Male Age : HGS0125427    Male            None  37   
2  Gender : Female Age : HGS0125435  Female  BHUMIKA DARNAL  32   
3                                 1    Male     RONGSENWATI  46   
4                                 1  Female     SENTIMONGLA  37   

  Father's/Husband's Name    Voter Id  
0               CHUBAWATI  HGS0073544  
1                    None  HGS0125427  
2       BIR BA